In [ ]:
import os
from flask import Flask, render_template
import paho.mqtt.client as mqtt
import threading
import matplotlib.pyplot as plt

app = Flask(__name__)

temperature_values = []
humidity_values = []
fan_state = "On"
window_state = "Open"
led_state="On"

def on_connect(client, userdata, flags, rc):
    print("Connected with result code " + str(rc))
    client.subscribe("temperature")
    client.subscribe("humidity")
    client.subscribe("fan_state")
    client.subscribe("window_state")
    client.subscribe("led_state")
    

def on_message(client, userdata, msg):
    global temperature_values, humidity_values, fan_state, window_state, led_state
    if msg.topic == "temperature":
        temperature_values.append(float(msg.payload))
    elif msg.topic == "humidity":
        humidity_values.append(float(msg.payload))
    elif msg.topic == "fan_state":
        fan_state = msg.payload.decode()
    elif msg.topic == "window_state":
        window_state = msg.payload.decode()
    elif msg.topic == "led_state":
        led_state = msg.payload.decode()

        
client = mqtt.Client()
client.on_connect = on_connect
client.on_message = on_message

def mqtt_thread():
    client.connect("192.168.137.1", 1883, 60)
    client.loop_forever()

mqtt_thread = threading.Thread(target=mqtt_thread)
mqtt_thread.start()

# Create the static directory if it doesn't exist
static_dir = os.path.join(os.getcwd(), 'static')
os.makedirs(static_dir, exist_ok=True)

@app.route('/')
def index():
    global temperature_values, humidity_values, fan_state, window_state, led_state

    # Update the button text based on the current state
    fan_button_text = "Off" if fan_state == "Off" else "On"
    window_button_text = "Closed" if window_state == "Closed" else "Open"
    led_button_text = "Off" if led_state == "Off" else "On"

    return render_template('dashboard.html', 
                           current_temperature=temperature_values[-1] if temperature_values else None,
                           current_humidity=humidity_values[-1] if humidity_values else None,
                           max_temperature=max(temperature_values) if temperature_values else None,
                           min_temperature=min(temperature_values) if temperature_values else None,
                           max_humidity=max(humidity_values) if humidity_values else None,
                           min_humidity=min(humidity_values) if humidity_values else None,
                           fan_state=fan_state,
                           window_state=window_state,
                           led_state=led_state,
                           fan_button_text=fan_button_text,
                           window_button_text=window_button_text,
                           led_button_text=led_button_text
                           )


@app.route('/plot')
def plot():
    global temperature_values, humidity_values
    # Plot temperature
    plt.figure(figsize=(10, 5))
    plt.plot(temperature_values, color='red')
    plt.title('Temperature Plot')
    plt.xlabel('Time')
    plt.ylabel('Temperature (°C)')
    temperature_plot_path = os.path.join('static', 'temperature_plot.png')
    plt.savefig(temperature_plot_path)
    plt.close()

    # Plot humidity
    plt.figure(figsize=(10, 5))
    plt.plot(humidity_values, color='blue')
    plt.title('Humidity Plot')
    plt.xlabel('Time')
    plt.ylabel('Humidity (%)')
    humidity_plot_path = os.path.join('static', 'humidity_plot.png')
    plt.savefig(humidity_plot_path)
    plt.close()

    return render_template('plot.html', temperature_plot_path=temperature_plot_path, humidity_plot_path=humidity_plot_path)

if __name__ == '__main__':
    app.run(debug=True, use_reloader=False)


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


C:\Users\DELL\AppData\Local\Temp\ipykernel_2984\360981865.py:38: DeprecationWarning: Callback API version 1 is deprecated, update to latest version
  client = mqtt.Client()
 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


Connected with result code 0


127.0.0.1 - - [19/May/2024 10:43:32] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [19/May/2024 10:43:32] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [19/May/2024 10:43:35] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [19/May/2024 10:44:04] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [19/May/2024 10:44:05] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [19/May/2024 10:44:21] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [19/May/2024 10:44:35] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [19/May/2024 10:44:36] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [19/May/2024 10:44:38] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [19/May/2024 10:44:40] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [19/May/2024 10:44:42] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [19/May/2024 10:44:43] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [19/May/2024 10:44:44] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [19/May/2024 10:44:45] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [19/May/2024 10:52:58] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [19/May/2024 11:19:38] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [19/May/2024 11

Connected with result code 0
